# Construction d'un système de recommandation

Nous avons décidé d'orienter notre projet sur la recommendation de films.
En effet durant ce confinement, nous avons eu le temps de visionner beaucoup de films,
mais nous nous sommes rendus compte que nous passions quasiment autant de temps
à choisir le film qu'à le regarder. D'où la nécessité de créer un système de re-
commendations afin d'optimiser notre temps de visionnage.
Nous avons chercher une base de données assez exploitable afin de mener à bien
notre projet. Nous nous sommes basés sur la base de données de 'The Movies Dataset'.


In [135]:
import numpy as np
import pandas as pd
import math
import re

## Fetching and cleaning data

Nous utilisons deux tables de données. L'une, *movies_metadata.csv*, contient une liste de films et des informations relativesau genre, date de sortie etc. 

### Informations sur les films

In [136]:
movies = pd.read_csv("movies_metadata.csv")
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [12]:
movies.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [13]:
def filter_correct_id(word):
    if re.fullmatch(r'[0-9]+', word):
        return word
    return "wrong_id"

In [14]:
# don't re-run
movies = movies[~movies.id.duplicated()]
movies.id = movies.id.apply(filter_correct_id)
movies = movies[movies.id != "wrong_id"]
movies.id = movies.id.astype('int64')

### Avis des utilisateurs

In [15]:
ratings = pd.read_csv("ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [16]:
# ne pas re-run !
ratings = ratings.drop(columns=['timestamp'])
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [17]:
ratings[(ratings['userId'] == 1) & (ratings['movieId'] == 31)]

,userId,movieId,rating
0,1,31,2.5


In [18]:
print(min(ratings.rating), max(ratings.rating))
ratings.describe()
ratings.dtypes

0.5 5.0


userId       int64
movieId      int64
rating     float64
dtype: object

In [19]:
nbPers = len(ratings.userId.unique())
nbMovi = len(ratings.movieId.unique())

## User Based Recommandation

Pour ce système, nous n'avons que besoin des notations des utilisateurs et des titres des films associés. Nous allons translater les notes afin que la moyenne des notes pour chaque utilisateur se trouve à 0. Par abus de langage nous appelons ces nouvelles notes les notes *normalisées*. 

In [141]:
movies_title = movies.loc[movies['id'].isin(ratings.movieId), ['id', 'title']]
print(len(ratings.movieId.unique()))
ratings_small = ratings.loc[ratings['userId'] <= 50]

9066


In [186]:
def mean_user(uid):
    '''
    Retourne la moyenne des notes données par l'utilisateur d'id uid
    '''
    n = ratings.loc[ratings['userId'] == uid].count().loc['userId']
    s = ratings.loc[ratings['userId'] == uid].sum().loc['rating']
    return s / n

In [187]:
def normalize_user(df):
    '''
    Ajoute une colonne dans la dataframe df contenant les notes normalisées des utilisateurs
    '''
    mean = df.loc[:, ['userId']].drop_duplicates()
    mean['mu'] = mean['userId'].map(lambda uid : mean_user(uid))
    mean = mean.set_index('userId')
    df['rating_norm'] = df[['userId', 'rating']].apply(lambda row : row['rating'] -  mean.loc[int(row['userId'])]['mu'], axis=1)

In [188]:
normalize_user(ratings_small)
print(ratings_small.head())

   userId  movieId  rating  rating_norm
0       1       31     2.5        -0.05
1       1     1029     3.0         0.45
2       1     1061     3.0         0.45
3       1     1129     2.0        -0.55
4       1     1172     4.0         1.45


/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Regrouper les utilisateurs dans des peer-group

Pour déterminer si deux utilisateurs se ressemblent en termes de goûts, nous utilisons un taux de corrélation sur les avis données. Nous allons comparer quatres taux de corrélations différents. Le premier ```cor()``` calcule le taux de corrélation classique donné par la formule :
$$
cor(u, v) = \frac{\sum_{k \in I_{uv}} s_{uk} s_{vk}}{\sqrt{\sum_{k \in I_{uv}} s_{uk}^2}\sqrt{\sum_{k \in I_{uv}} s_{vk}^2}}
$$

Le taux de corrélation ajusté ```cor_adj()``` permet de ne pas donner trop d'importance aux films populaires que beaucoup de personnes ont vu.

Le taux de correlation calculé par ```cor_dis()``` permet de ne pas donner une correlation trop élevée si les deux utilisateurs n'ont pas donné assez d'avis sur des films en commun. 

Enfin la fonction ```cor_dis_adj()``` fait un mélange des deux dernières amélioration : il filtre les films trop populaire et n'apporte de l'importance seulement si deux personnes ont données leur avis sur un certain nombre de films.

In [24]:
def cor(u, v, df, Iuv):
    su = df.loc[(df['userId'] == u) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    sv = df.loc[(df['userId'] == v) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    su = np.array(su)
    sv = np.array(sv)
    
    return np.dot(su, sv) / math.sqrt(np.dot(su, su) * np.dot(sv, sv))

In [25]:
def cor_adj(u, v, df, Iuv):
    nb_rat = df.loc[:, ['movieId', 'rating']].groupby(['movieId']).count()
    
    sum_up = 0
    sum_down_u = 0
    sum_down_v = 0
    for movie in Iuv.movieId.unique() :
        suk = df.loc[(df['userId'] == u) & (df['movieId'] == movie), ['rating_norm']]
        svk = df.loc[(df['userId'] == v) & (df['movieId'] == movie), ['rating_norm']]
        suk, svk = float(suk), float(svk)
        
        sum_up += suk * svk / nb_rat.at[movie, 'rating']
        sum_down_u += suk**2 /  nb_rat.at[movie, 'rating']
        sum_down_v += svk**2 /  nb_rat.at[movie, 'rating']
    return sum_up / math.sqrt(sum_down_u * sum_down_v)

In [26]:
def cor_dis(u, v, df, Iuv):
    beta = 20
    correlation = cor(u, v, df, Iuv)
    return correlation * min(len(Iuv), beta)/beta

In [27]:
def cor_dis_adj(u, v, df, Iuv):
    beta = 20
    correlation = cor_adj(u, v, df, Iuv)
    return correlation * min(len(Iuv), beta)/beta

**Test des taux de corrélation sur les utilisateurs 2 et 3 qui ont 8 films en communs**

In [28]:
u, v = 2, 3
df = ratings_small
Iu = df.loc[df['userId'] == u, ['movieId']]
Iv = df.loc[df['userId'] == v, ['movieId']]
Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')
print(Iuv)

print(cor(u, v, df, Iuv))
# print(cor_adj(u, v, df))
print(cor_dis(u, v, df, Iuv))
# print(cor_dis_adj(u, v, df))

    movieId
27      110
49      296
57      356
64      377
79      527
88      588
91      592
92      593
-0.016060945830838957
-0.006424378332335582


Nous construisons maintenant la matrice de correlation. Puisqu'une telle matrice est symétrique, nous avons préféré utiliser une dataframe à deux entrées et ne stocker la corrélation pour un couple qu'une seule fois.

In [209]:
def uu_matrix(df, cor_fct=cor):
    '''
    Retourne la dataframe des taux de corrélations des utilisateurs de df
    '''
    correlation = []
    users = df.userId.unique()
    couples = []
    for i in range(len(users)):
        u = users[i]
        if not u % 20 : 
            print('user:', u, end='')
        for j in range(i + 1, len(users)):
            v = users[j]
            Iu = df.loc[df['userId'] == u, ['movieId']]
            Iv = df.loc[df['userId'] == v, ['movieId']]
            Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')
            if Iuv.size :
                couples.append((u, v))
                correlation.append(cor_fct(u, v, df, Iuv))
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    cor = pd.Series(correlation, index=index)
    return cor

In [145]:
def get_cor(u, v, cm):
    '''
    Retourne le taux de correlation entre u et v stocké dans cm
    '''
    if u > v :
        u, v = v, u
    index = list(cm.index.values)
    if (u, v) in index :
        return float(cm.loc[(u, v)])
    return float('-inf')

In [163]:
cm_small = uu_matrix(ratings_small)

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


user :  20user :  40

In [164]:
cm_small.head()

u  v 
1  4     0.042137
   5    -1.000000
   7    -0.752427
   9     1.000000
   15    0.043773
dtype: float64

In [168]:
print(get_cor(1, 4, cm_small))
print(get_cor(4, 1, cm_small))
print(get_cor(2, 1, cm_small))

0.042136808375910856
0.042136808375910856
-inf


Nous allons maintenant prédire la note qu'un utilisateur **u** donnerait à un film *m . Pour cela, nous allons faire la somme des notes données à l'item *i* par les k utilisateurs plus proches de u qui ont donné une note à m. Cette somme sera pondérée par les coéfficients de corrélations.

$$
\hat{\sigma}_{um} = \mu_u + \frac{\sum_{v \in P_u(m)} s_{vm} \cdot cor(u, v)}{\sum_{v \in P_u(m)} |cor(u,v)|}
$$

Le peer-group de l'utilisateur u pour le film m est l'ensemble des k utilisateurs qui ont donné une note au film m les plus proche de l'utilisateur u en terme de taux de corrélation. 

In [36]:
def peers(user, movie, k, cm, df):
    '''
    Retourne les k utilisateurs du peer-group de (user, movie)
    '''
    top = [(float('-inf'), user)] * k
    df_movie = df.loc[df['movieId'] == movie, ['userId', 'rating_norm']]
    for v in df_movie.userId.unique():
        taux = get_cor(user, v, cm)
        if taux and taux > top[-1][0] :
            top += [(taux, v)]
            top.sort(reverse=True)
            top = top[:-1]
    return [t[1] for t in top]

In [473]:
df = ratings_small
cm = cm_small
movie = 10
u = 85
k = 4

others = df.loc[df['movieId'] == movie].userId.tolist()
print('others: ',others)
top = cm.xs(user)
print('top:', top.head())
top = top.loc[top.index.isin(others)]
print('top:', top.head())
top = top.sort_values(by=['correlation'], ascending=False)
print('top k: ', top[:k])

print(cm.loc[(u, 85)])
print('core', get_cor(u, 85, cm))
print('ligne:\n', df.loc[(df['userId'] == 85) & (df['movieId'] == movie), :])
print(df.head())

others:  [2, 4, 7, 15, 19, 21, 39, 47, 50, 56, 73, 74, 77, 78, 80, 82, 85, 92]
top:     correlation
v              
4      0.042137
5     -1.000000
7     -0.752427
9      1.000000
15     0.043773
top:     correlation
v              
4      0.042137
7     -0.752427
15     0.043773
19     0.053458
21    -0.478805
top k:      correlation
v              
39     1.000000
19     0.053458
15     0.043773
4      0.042137


TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [85] of <class 'int'>

In [ ]:
def peers2(user, movie, k, cm, df):
    '''
    Retourne les k utilisateurs du peer-group de (user, movie)
    '''
    # list of users that rated movie
    others = df.loc[df['movieId'] == movie].userId.tolist()
    
    a = cm.xs(user, level='v').rename_axis(index=['v'])
    b = cm.xs(user, level='u')
    correlation = a.append(b)
    
    
    
    for v in df_movie.userId.unique():
        taux = get_cor(user, v, cm)
        if taux and taux > top[-1][0] :
            top += [(taux, v)]
            top.sort(reverse=True)
            top = top[:-1]
    return [t[1] for t in top]

In [520]:
df = ratings_small
cm = cm_small
movie = 10
u = 2
k = 4

# print(cm.loc[(slice(None), 6), :])
# print(cm.xs(6, level='v'))
# print(cm.xs(6, level='u').head())
# print(cm.xs(6, level='v').rename_axis(index=['v']).head())
# a = cm.xs(6, level='u')
# b = cm.xs(6, level='v').rename_axis(index=['v'])
# a.append(b)
# print(a.tail())


others = df.loc[df['movieId'] == movie].userId.tolist()
print(others)

print(u in cm.index.get_level_values(1))

print(cm.head())

# a = cm.xs(u, level='v').rename_axis(index=['v'])
# b = cm.xs(u, level='u')
# top = a.append(b)
    
# print(top.head())

# print()
# top = top.loc[top.index.isin(others)]
# print(top.head())

# top = top.sort_values(by=['correlation'], ascending=False)
# print(top.head())

# print(top[:k].index.tolist())
# print(peers(u, movie, k, cm, df))

[2, 4, 7, 15, 19, 21, 39, 47, 50, 56, 73, 74, 77, 78, 80, 82, 85, 92]
False
      correlation
u v              
1 4      0.042137
  5     -1.000000
  7     -0.752427
  9      1.000000
  15     0.043773


In [250]:
def predict(user, movie, cm, df):
    mu = mean_rating(user)
    peer_group = peers(user, movie, 4, cm, df)
    sum_up, sum_down = 0, 0
    for v in peer_group:
        svm = float(df.loc[(df['userId'] == v) & (df['movieId'] == movie), 'rating_norm'])
        sum_up += svm * get_cor(user, v, cm)
        sum_down += abs(get_cor(user, v, cm))
    sum_down = 1 if sum_down == 0 else sum_down
    return mu + sum_up / sum_down

In [251]:
friends = peers(85, 10, 4, cm_small, ratings_small)
p = predict(1, 10, cm_small, ratings_small)
print(friends)
print(p)
print(df.loc[(df['userId'].isin(friends)) & (df['movieId'] == 10)])

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


TypeError: cannot convert the series to <class 'float'>

## Item Based Recommandation

Nous allons maintenant construire un système de recommandation basé sur les items. Dans le système user-based, pour prédire la note donnée par l'utilisateur *user* au film *movie*, on regardait les notes donnés à ce même film *movie* par des utilisateurs similaires à *user*. De manière analogue, dans le système item-based, pour prédire la note donnée par user à *movie*, on regarde les notes données par *user* à des films similaires à *movie*.

Pour cela, nous allons réecrire toutes les fonctions précédentes pour qu'elles soient flexibles à une utilisation user-based ou item-based. (idée pour factoriser le code)

In [97]:
ratings_mm = ratings.loc[ratings['movieId'] <= 100]

**Normalisation des notes données aux films par leur moyenne**

In [98]:
def mean_movie(mid):
    '''
    Retourne la moyenne des notes données au film d'id mid
    '''
    n = ratings.loc[ratings['movieId'] == mid].count().loc['movieId']
    s = ratings.loc[ratings['movieId'] == mid].sum().loc['rating']
    return s / n

In [99]:
def normalize_movie(df):
    '''
    Ajoute une colonne dans la dataframe df contenant les notes normalisées des films
    '''
    mean = df.loc[:, ['movieId']].drop_duplicates()
    mean['mu'] = mean['movieId'].map(lambda mid : mean_movie(mid))
    mean = mean.set_index('movieId')
    df['norm_movie'] = df[['movieId', 'rating']].apply(lambda row : row['rating'] -  mean.loc[int(row['movieId'])]['mu'], axis=1)

In [100]:
normalize_movie(ratings_mm)

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [101]:
ratings_mm.head()

,userId,movieId,rating,norm_movie
0,1,31,2.5,-0.678571
20,2,10,4.0,0.549180
21,2,17,5.0,1.075581
22,2,39,5.0,1.450000
23,2,47,4.0,-0.034826


In [102]:
def cor_movie(u, v, df, Iuv):
    su = df.loc[(df['movieId'] == u) & (df['userId'].isin(Iuv['userId']))].norm_movie
    sv = df.loc[(df['movieId'] == v) & (df['userId'].isin(Iuv['userId']))].norm_movie
    su = np.array(su)
    sv = np.array(sv)
    
    return np.dot(su, sv) / math.sqrt(np.dot(su, su) * np.dot(sv, sv))

In [119]:
def mm_matrix(df, cor_fct=cor_movie):
    '''
    Retourne la dataframe des taux de corrélations des utilisateurs de df
    '''
    correlation = []
    movies = df.movieId.unique()
    couples = []
    for i in range(len(movies)):
        u = movies[i]
#         if not i % 20 : 
        print('movie number: ', i, end=' ')
        for j in range(i + 1, len(movies)):
            v = movies[j]
            Iu = df.loc[df['movieId'] == u, ['userId']]
            Iv = df.loc[df['movieId'] == v, ['userId']]
            Iuv = Iu.join(Iv.set_index('userId'), on='userId', how='inner')
            if Iuv.size :
                couples.append((u, v))
                correlation.append(cor_fct(u, v, df, Iuv))
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    cor = pd.DataFrame(correlation, index=index, columns=['correlation'])
    return cor

In [120]:
def get_cor(u, v, cm):
    '''
    Retourne le taux de correlation entre u et v stocké dans cm
    '''
    if u > v :
        u, v = v, u
    index = list(cm.index.values)
    if (u, v) in index :
        return float(cm.loc[(u, v)])
    return float('-inf')

In [121]:
cm_small = mm_matrix(ratings_mm)

movie number:  0 

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


movie number:  1 movie number:  2 movie number:  3 movie number:  4 movie number:  5 movie number:  6 movie number:  7 movie number:  8 movie number:  9 movie number:  10 movie number:  11 movie number:  12 movie number:  13 movie number:  14 movie number:  15 movie number:  16 movie number:  17 movie number:  18 movie number:  19 movie number:  20 movie number:  21 movie number:  22 movie number:  23 movie number:  24 movie number:  25 movie number:  26 movie number:  27 movie number:  28 movie number:  29 movie number:  30 movie number:  31 movie number:  32 movie number:  33 movie number:  34 movie number:  35 movie number:  36 movie number:  37 movie number:  38 movie number:  39 movie number:  40 movie number:  41 movie number:  42 movie number:  43 movie number:  44 movie number:  45 movie number:  46 movie number:  47 movie number:  48 movie number:  49 movie number:  50 movie number:  51 movie number:  52 movie number:  53 movie number:  54 movie number:  55 movie number:  56 m

In [122]:
cm_small.head()

correlation
u  v              
31 10     0.424558
   17     0.286119
   39     0.528895
   47    -0.040864
   50     0.381653

In [132]:
def peers_movie(user, movie, k, cm, df):
    '''
    Retourne les k films du peer-group de (movie, user)
    '''
    top = [(float('-inf'), movie)] * k
    df_user = df.loc[df['userId'] == user, ['movieId', 'norm_movie']]
    for v in df_user.movieId.unique():
        taux = get_cor(movie, v, cm)
        if taux and taux > top[-1][0] :
            top += [(taux, v)]
            top.sort(reverse=True)
            top = top[:-1]
    return [t[1] for t in top]

In [133]:
def predict(user, movie, cm, df):
    res = mean_movie(movie)
    peer_group = peers_movie(user, movie, 4, cm, df)
    for v in peer_group:
        print(df.loc[(df['movieId'] == v) & (df['userId'] == user), 'norm_movie'])
#         svu = float(df.loc[(df['movieId'] == v) & (df['userId'] == user), 'norm_movie'])
#         res += svm * get_cor(movie, v, cm) / abs(get_cor(movie, v, cm))
    return res

In [134]:
cm = cm_small
df = ratings_mm
user = 1
movie = 10
k = 4

friends = peers_movie(user, movie, k, cm, df)
p = predict(user, movie, cm, df)
print(friends)
print(p)
print(df.loc[(df['movieId'].isin(friends)) & (df['userId'] == user)])

Series([], Name: norm_movie, dtype: float64)
Series([], Name: norm_movie, dtype: float64)
Series([], Name: norm_movie, dtype: float64)
Series([], Name: norm_movie, dtype: float64)
[10, 10, 10, 10]
3.4508196721311477
Empty DataFrame
Columns: [userId, movieId, rating, norm_movie]
Index: []
